In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Baixa certidoes criminais de candidatos a deputados federais nas eleições de 2022
# Fonte: https://divulgacandcontas.tse.jus.br/divulga/#/
# Hint: https://stackoverflow.com/questions/73506117/with-python-and-selenium-how-to-find-the-hidden-links-of-files-on-a-website
# Usado Ubuntu 22.04.1 LTS
#

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time as t
import os
import pandas as pd
import requests
import shutil

In [11]:
# Chrome setup
chrome_options = Options()
# http://www.google.com/googlebooks/chrome/med_26.html
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument("window-size=1280,720")


In [ ]:
# Mostra diretório atual e seta um novo apenas para downloads

In [12]:
os.getcwd()

'/home/abraji/Documentos/Code/pinpoint/repo/resultados/certidoes'

In [13]:
os.chdir('/home/abraji/Documentos/Code/pinpoint/repo/resultados/certidoes')

In [14]:
url_raiz = 'https://divulgacandcontas.tse.jus.br/divulga/#/candidato/2022/2040602022'

In [15]:
'''
Acre (AC)
Alagoas (AL)
Amazonas (AM)
Bahia (BA)
Ceará (CE)
Distrito Federal (DF)
Espírito Santo (ES)
Goiás (GO)
Maranhão (MA)

Mato Grosso (MT)
Mato Grosso do Sul (MS)
Minas Gerais (MG)

Pará (PA)
Paraíba (PB)
Paraná (PR)
Pernambuco (PE)

Piauí (PI)
Rio Grande do Norte (RN)
Rio Grande do Sul (RS)
Rondônia (RO)

Roraima (RR)
Santa Catarina (SC)
Sergipe (SE)
Tocantins (TO)

'''

'\nAcre (AC)\nAlagoas (AL)\nAmazonas (AM)\nBahia (BA)\nCeará (CE)\nDistrito Federal (DF)\nEspírito Santo (ES)\nGoiás (GO)\nMaranhão (MA)\n\nMato Grosso (MT)\nMato Grosso do Sul (MS)\nMinas Gerais (MG)\n\nPará (PA)\nParaíba (PB)\nParaná (PR)\nPernambuco (PE)\n\nPiauí (PI)\nRio Grande do Norte (RN)\nRio Grande do Sul (RS)\nRondônia (RO)\n\nRoraima (RR)\nSanta Catarina (SC)\nSergipe (SE)\nTocantins (TO)\n\n'

In [16]:
# Lista de candidatos:
# https://dadosabertos.tse.jus.br/dataset/candidatos-2022/resource/435145fd-bc9d-446a-ac9d-273f585a0bb9
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'latin_1'}
candidatos = pd.read_csv("/home/abraji/Documentos/Code/pinpoint/repo/consulta_cand_2022_BRASIL_24_ago.csv", **kwargs)
# Exemplo de baixar 4 UFs
busca = candidatos[((candidatos['SG_UF'] == 'RR') |
                    (candidatos['SG_UF'] == 'SC') |
                    (candidatos['SG_UF'] == 'SE') |
                    (candidatos['SG_UF'] == 'TO') 
                    ) &
                   (candidatos['DS_CARGO'] == 'DEPUTADO FEDERAL') 
                  ]
busca.shape

(795, 71)

In [ ]:
# Iteração no dataframe
for num, row in busca.iterrows():
    # Cria nome do arquivo para cada candidato
    nome = (row['NM_CANDIDATO']).strip() + "_" + (row['SG_UF']).strip()  + "_" + (row['DS_CARGO']).strip()
    uf = (row['SG_UF']).strip()
    print(nome)
    print("-/-")
    # O ID sequencial vai criar a URL do candidato
    sequencial = (row['SQ_CANDIDATO']).strip()

    # Cria o endereço completo da página
    url_pag = url_raiz + '/' + uf + '/' + sequencial
    print(url_pag)
    
    webdriver_service = Service('/home/abraji/Documentos/Code/chromedriver_linux64/chromedriver') ## caminho para onde você salvou o binário do chromedriver
    browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)
    
    counter = 0

    browser.get(url_pag) 
    
    
    try:
        # Com Inspecionar elemento achei o CSS selector das certidões em cada página - nela estão os links
        links = WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".dvg-link-doc.dvg-certidao")))
    
    
        # Itera nos links e clica em cada um
        for x in range(len(links)):
            current_link = links[counter]
            print(current_link.text)
            t.sleep(1)
            current_link.click()
            t.sleep(1)
            browser.switch_to.window(browser.window_handles[-1])
            print(browser.current_url)
        
            link_baixa = browser.current_url
        
            # Faz a requisição do link do PDF
            r = requests.get(link_baixa,
                 stream=True, headers={'User-agent': 'Mozilla/5.0'})
        
            nome_final = nome + "_" + str(counter) + ".pdf"

            # Testa se o link está ok
            if r.status_code == 200:
                # Se estiver - write in binary mode
                with open(nome_final, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
        
            t.sleep(1)
            # Abre a página inicial do candidato da vez
            browser.get(url_pag) 
            # Adiciona no contador
            counter = counter + 1
            # captura o próximo link
            links = WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".dvg-link-doc.dvg-certidao")))
            t.sleep(1)
        
    except:
        pass
    # Fecha a página do candidato para abrir no próximo
    browser.close()